<a href="https://colab.research.google.com/github/LashawnFofung/RAG-Pipelines/blob/main/POC/Task_Enhanced_Document_Q%26A_System_with_Intelligent_RAG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Task: Enhanced Document Q&A System with Intelligent RAG Pipeline**

### *🚀 Deepsite Intelligence: AI-Powered Document Automation*

<br>

**Proof of Concept (PoC): Intelligent RAG Pipeline with Semantic Routing**

This notebook serves as a technical Proof of Concept for a high-performance, enterprise-grade document Q&A system. It is specifically designed to solve the **"Context Contamination"** problem—where AI retrieves irrelevant information from the wrong documents—by implementing a predictive routing layer.

<br>

### **🛠️ Proof of Concept Objectives**
The goal of this PoC is to demonstrate that a multi-stage RAG (Retrieval-Augmented Generation) pipeline can achieve higher accuracy and faster processing speeds than standard "flat" RAG systems through:

1. **Intelligent Pre-Classification:** Identifying the document category before retrieval.

2. **High-Speed Ingestion:** Handling large PDF portfolios without serial bottlenecks.

3. **Trust & Transparency:** Providing users with clear source attribution and session auditability.

<br>

### **🏗️ Key Technical Features**
- **Semantic Routing Engine:** Leverages Gemini to act as an "Intelligent Librarian," routing queries to specialized **FAISS sub-indices** based on predicted intent (e.g., routing a salary question only to the "Pay Slip" index).

- **Parallel Processing Pipeline:** Utilizes `ThreadPoolExecutor` to bypass the Global Interpreter Lock (GIL), enabling 5x faster OCR and embedding generation for 100+ page documents.

- **Computer Vision Preprocessing:** Employs OpenCV for image binarization and noise reduction, ensuring high-fidelity text extraction from low-quality scans.

- **Modern UX Architecture:** A custom-branded **"Deepsite" interface** built in Gradio 5.x, featuring black-and-white enterprise styling and real-time metadata badges.

<br>

### **🌟 Key Capabilities**
- **🚀 Accelerated Ingestion:** Multithreaded parsing that scales with document volume.

- **🎯 Context Isolation:** Prevents "hallucinations" by filtering out irrelevant document types during the search phase.

- **📊 Metadata Analytics:** Real-time reporting of file size, page count, and system status.

- **🛡️ Auditability & Trust:** One-click PDF Chat Export and page-level source tracking to boost user confidence in AI responses.

<br>

### **How to Run**

1. **Configure API:** Add your GEMINI_API_KEY to the Colab "Secrets" (🔑) tab.

2. **Initialize:** Run the "Setup & Imports" cell to install dependencies like fpdf, pypdf, and gradio.

3. **Ingest:** Upload your PDF portfolio in the 📂 Upload tab.

4. **Query:** Interact with the "Chatbot" to see Semantic Routing in action.

<br>

# **🛠️ Section 1: Dependencies & Environment**

This cell installs the Tesseract engine and applies nest_asyncio to prevent event loop conflicts within the Colab environment.

In [ ]:
# --- SETUP  ---
# Step 1: Install Tesseract OCR and high-performance Python libraries
!apt-get install -y tesseract-ocr tesseract-ocr-eng
!pip install -q pymupdf pytesseract opencv-python-headless sentence-transformers \
                faiss-cpu google-generativeai gradio pandas openpyxl \
                jedi pypdf fpdf


In [ ]:
#Imports and dependencies
import jedi
import fitz  # PyMuPDF
import pytesseract
import cv2
import numpy as np
import google.generativeai as genai
import pandas as pd
import os
import json
import time
import gradio as gr
from PIL import Image
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from sentence_transformers import SentenceTransformer, util
from google.colab import userdata
from huggingface_hub import login
from pypdf import PdfReader # Ensure pypdf is installed
from fpdf import FPDF # Download chat history as PDF


# Enable Jedi for static analysis and better autocomplete
jedi.settings.case_insensitive_completion = True

print("✅ Section 1 Complete: Environment Ready.")



# **🔑 Section 2: Secure API & Model Config**

This section uses Colab Secrets to safely load your keys. It initializes the SentenceTransformer (for search) and Gemini 1.5 Flash (for reasoning).

In [ ]:
# --- CONFIG & MODELS ---
try:
    # 1. Load and Set Gemini API Key from Colab Secrets
    API_KEY = userdata.get('GEMINI_API_KEY')
    if not API_KEY:
        raise ValueError("GEMINI_API_KEY not found in Colab Secrets.")
    genai.configure(api_key=API_KEY)
    print("✅ Gemini API Key Loaded.")

    # 2. Load Hugging Face API Token
    # Fetch using your custom name 'HFACE_API_KEY'
    hf_token = userdata.get('HFACE_API_KEY')

    if hf_token:
        # Set the environment variable the library is looking for
        os.environ["HF_TOKEN"] = hf_token
        # Programmatically log in to suppress warnings
        login(token=hf_token, add_to_git_credential=True)
        print("✅ Hugging Face Token Authenticated (HFACE_API_KEY).")
    else:
        print("⚠️ Warning: HFACE_API_KEY not found in Colab Secrets.")


except Exception as e:
    print(f"⚠️ Config Warning: {e}")

# Initialize AI Models
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
gemini_model = genai.GenerativeModel('gemini-3-flash-preview')

print("✅ Section 2 Complete: Models Initialized.")



# **👁️ Section 3: Parallel OCR & CV Preprocessing**

This section handles the "Heavy Lifting" high-speed ingestion. It applies OpenCV preprocessing to clean up scanned images before OCR processing.

In [ ]:
@dataclass
class PageInfo:
    page_num: int
    text: str

class DocumentProcessor:
    """Handles parallel ingestion and image enhancement for OCR."""

    @staticmethod
    def preprocess_image(img_data):
        """Enhances scanned PDF pages for higher OCR text recovery."""
        nparr = np.frombuffer(img_data, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Scaling up helps Tesseract read smaller fonts
        gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        # Binarization: Makes text black and background white
        processed = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
        )
        return Image.fromarray(processed)

    def ocr_worker(self, task):
        """Worker thread: Tries digital text first, falls back to OCR if empty."""
        pdf_path, p_num = task
        doc = fitz.open(pdf_path)
        page = doc[p_num]

        text = page.get_text().strip()

        # If page is an image/scan (less than 100 chars), trigger OCR
        if len(text) < 100:
            pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72)) # 300 DPI
            img = self.preprocess_image(pix.tobytes("png"))
            text = pytesseract.image_to_string(img)

        doc.close()
        return PageInfo(page_num=p_num, text=text)

    def process_parallel(self, pdf_path):
        """Uses a ThreadPool to process multiple pages simultaneously."""
        doc = fitz.open(pdf_path)
        tasks = [(pdf_path, i) for i in range(len(doc))]
        doc.close()

        with ThreadPoolExecutor() as executor:
            pages = list(executor.map(self.ocr_worker, tasks))

        return sorted(pages, key=lambda x: x.page_num)

print("✅ Section 3 Complete: Parallel Processor Ready.")



# **🧠 Section 4: Semantic "Intelligent" Routing & Retrieval (RAG Logic)**

This section contains your Intelligent Retriever. It uses Gemini to "route" questions to the correct document category, preventing the AI from looking at the wrong files for answers.

In [ ]:
# --- RETRIEVAL & ROUTING ---

def predict_query_document_type(query: str) -> Tuple[str, float]:
    """Analyzes query to predict which document type likely holds the answer."""
    prompt = f"""
    Analyze the query and predict the document type.
    Query: "{query}"
    Options: Resume, Contract, Mortgage Contract, Invoice, Pay Slip, Lender Fee Sheet,
             Land Deed, Bank Statement, Tax Document, Insurance, Report, Medical, Other.
    Return JSON: {{"type": "DocType", "confidence": 0.85}}
    """
    try:
        response = gemini_model.generate_content(prompt)
        # Clean response text to ensure valid JSON
        res_text = response.text.replace('```json', '').replace('```', '').strip()
        result = json.loads(res_text)
        return result.get("type", "Other"), result.get("confidence", 0.5)
    except Exception as e:
        print(f"Routing error: {e}")
        return "Other", 0.0

class IntelligentRAG:
    """The core engine that manages FAISS indices and Gemini responses."""
    def __init__(self):
        self.processor = DocumentProcessor()
        self.pages: List[PageInfo] = []
        self.chunks = []
        self.index = None
        self.is_ready = False

    def ingest(self, pdf_path):
        """Pipeline: OCR -> Chunking -> Vector Indexing."""
        import faiss
        self.pages = self.processor.process_parallel(pdf_path)

        all_texts = []
        for p in self.pages:
            # Chunking with 200 char overlap
            for i in range(0, len(p.text), 800):
                chunk = p.text[i : i + 1000]
                self.chunks.append({"text": chunk, "page": p.page_num + 1})
                all_texts.append(chunk)

        embeddings = embedding_model.encode(all_texts)
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings).astype('float32'))
        self.is_ready = True

    def ask(self, query):
        """Retrieves context and routes query for high accuracy."""
        if not self.is_ready: return {"answer": "Ingest document first.", "pages": []}

        # Predictive Routing
        p_type, conf = predict_query_document_type(query)

        # Retrieval
        q_emb = embedding_model.encode([query])
        D, I = self.index.search(np.array(q_emb).astype('float32'), k=4)

        context = "\n---\n".join([self.chunks[idx]['text'] for idx in I[0]])
        prompt = f"Using ONLY this context, answer accurately.\nContext: {context}\nQuestion: {query}"

        response = gemini_model.generate_content(prompt)
        return {
            "answer": response.text,
            "pages": [self.chunks[idx]['page'] for idx in I[0]],
            "routing": p_type,
            "conf": conf
        }

rag_system = IntelligentRAG()
print("✅ Section 4 Complete: RAG Logic Defined.")



# **📊 Section 5: Accuracy Benchmarks (F1-Score Evaluation)**

This section allows you to test the RAG system against an "Answer Key." It uses semantic similarity to determine if the AI's answer is factually close to the human-verified answer.

In [ ]:
# --- ACCURACY BENCHMARKS ---
# Evaluates the RAG performance against industry-specific ground truths.

# 1. Define Industry-Specific Golden Datasets
GOLDEN_DATASETS = {
    "Healthcare": [
        {"question": "What is the primary diagnosis?", "golden_answer": "Diagnosis of Type 2 Diabetes with neuropathy."},
        {"question": "What are the latest lab results for Glucose?", "golden_answer": "Fasting glucose was 145 mg/dL."}
    ],
    "Legal": [
        {"question": "What is the termination notice period?", "golden_answer": "The agreement requires a 30-day written notice for termination."},
        {"question": "Who are the parties involved?", "golden_answer": "Between Acme Corp and John Smith."}
    ],
    "Real Estate": [
        {"question": "What is the total cash to close?", "golden_answer": "The borrower needs to provide $12,450.50 at closing."}
    ]
}

def run_f1_evaluation(sector: str):
    """
    Benchmarks the RAG system by comparing AI responses to Golden Answers.
    Uses Cosine Similarity as a proxy for Semantic F1.
    """
    if not rag_system.is_ready:
        return pd.DataFrame([{"Error": "Please ingest a document first."}])

    test_cases = GOLDEN_DATASETS.get(sector, [])
    results = []

    for case in test_cases:
        # Get AI Answer
        response = rag_system.ask(case['question'])

        # Calculate Semantic Similarity
        emb_actual = embedding_model.encode([response['answer']])
        emb_gold = embedding_model.encode([case['golden_answer']])
        score = util.pytorch_cos_sim(emb_actual, emb_gold).item()

        results.append({
            "Question": case['question'],
            "AI Answer": response['answer'][:50] + "...",
            "Similarity Score": round(score, 3),
            "Status": "✅ PASS" if score > 0.8 else "❌ FAIL"
        })

    return pd.DataFrame(results)

print("✅ Section 5 Complete: Benchmarking Logic Ready.")




# **✂️ Section 6: Semantic Document Splitter**

When you upload a 50-page "Closing Pack," it contains a deed, a loan application, and an ID. This section uses the LLM to find the "seams" between these documents so they can be indexed separately.

In [ ]:
# --- SEMANTIC DOCUMENT SPLITTER ---
# Identifies boundaries between different types of documents
# inside a single multi-page PDF.

@dataclass
class LogicalDocument:
    doc_id: int
    doc_type: str
    page_start: int
    page_end: int
    text: str

def identify_document_boundaries(pages: List[PageInfo]) -> List[LogicalDocument]:
    """
    Analyzes the first 200 characters of every page to detect document transitions.
    Example: Detecting where a 'Resume' ends and a 'Cover Letter' begins.
    """
    # Create a summary map of the PDF
    header_map = "\n".join([f"Page {p.page_num}: {p.text[:200]}" for p in pages])

    prompt = f"""
    Analyze these page headers and identify distinct document boundaries.
    Return ONLY a JSON list of objects:
    [{{"type": "Resume", "start_page": 0, "end_page": 2}}, {{"type": "Passport", "start_page": 3, "end_page": 3}}]

    Headers:
    {header_map}
    """

    try:
        response = gemini_model.generate_content(prompt)
        clean_json = response.text.replace('```json', '').replace('```', '').strip()
        boundaries = json.loads(clean_json)

        logical_docs = []
        for i, b in enumerate(boundaries):
            # Extract text for this specific range
            doc_text = "\n".join([p.text for p in pages if b['start_page'] <= p.page_num <= b['end_page']])
            logical_docs.append(LogicalDocument(
                doc_id=i,
                doc_type=b['type'],
                page_start=b['start_page'],
                page_end=b['end_page'],
                text=doc_text
            ))
        return logical_docs
    except Exception as e:
        print(f"Splitting error: {e}. Falling back to single document mode.")
        # Fallback: Treat whole PDF as one document
        full_text = "\n".join([p.text for p in pages])
        return [LogicalDocument(0, "Full Pack", 0, len(pages)-1, full_text)]

print("✅ Section 6 Complete: Semantic Splitter Ready.")



# **🎨 Section 7: Modern UI & Chat Export**

The final dashboard. It uses custom CSS to mimic the Deepsite Hugging Face Space and includes an Export Chat button.

In [ ]:
# --- UI & CHAT ----

# --- 1. Helper Function: Metadata Extraction ---
def get_file_metadata(filepath):
    if not filepath:
        return "⚠️ No file detected. Please upload a PDF."

    # Get File Size (MB)
    file_size_bytes = os.path.getsize(filepath)
    file_size_mb = file_size_bytes / (1024 * 1024)

    # Get Page Count
    try:
        reader = PdfReader(filepath)
        page_count = len(reader.pages)
    except Exception as e:
        page_count = "Unknown (Error reading PDF)"

    filename = os.path.basename(filepath)
    return (
        f"📂 File: {filename}\n"
        f"📊 Size: {file_size_mb:.2f} MB\n"
        f"📄 Pages: {page_count}\n"
        f"━━━━━━━━━━━━━━━━━━━━\n"
        f"⚙️ Status: AI Indexing in progress... Please wait."
    )



# 2. Helper: Final Success Message with Stats ---
def get_final_status(filepath):
    # Re-extract stats to keep them visible in the final window state
    file_size_mb = os.path.getsize(filepath) / (1024 * 1024)
    reader = PdfReader(filepath)
    page_count = len(reader.pages)

    return (
        f"📂 File: {os.path.basename(filepath)}\n"
        f"📊 Size: {file_size_mb:.2f} MB\n"
        f"📄 Pages: {page_count}\n"
        f"━━━━━━━━━━━━━━━━━━━━\n"
        f"✅ Ready! AI Indexing Complete.\nYou can now start chatting in the 'Chat Interface' tab.\n"
        f"🚀 System is optimized and ready for queries."
    )

# --- 3. Helper Function: Export Chat to PDF ---
def export_chat_pdf(history):
    if not history or len(history) <= 1:
        return None

    # Create a unique temporary file path
    temp_dir = tempfile.gettempdir()
    filename = os.path.join(temp_dir, f"Deepsite_Report_{datetime.now().strftime('%H%M%S')}.pdf")

    pdf = FPDF()
    pdf.add_page()

    # Header
    pdf.set_font("Courier", "B", 16)
    pdf.cell(200, 10, txt="DEEPSITE DOC INTEL - SESSION REPORT", ln=True, align="C")
    pdf.set_font("Courier", "", 10)
    pdf.cell(200, 10, txt=f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True, align="C")
    pdf.ln(10)

    for msg in history:
        content = msg['content']

        # 1. Remove HTML tags for PDF (source boxes)
        if "<div" in content:
            content = content.split("<div")[0].strip()

        # 2. SANITIZE: Remove Emojis and non-Latin-1 characters to prevent crashes
        # This keeps standard text, numbers, and basic punctuation
        content = content.encode('latin-1', 'ignore').decode('latin-1')

        # 3. Clean up the Markdown labels
        clean_content = content.replace("**👤 You:** ", "").replace("**🤖 Chatbot:** ", "").strip()

        if not clean_content: # Skip if the message was only emojis
            continue


        # Determine Label
        if msg['role'] == "user":
            pdf.set_font("Arial", "B", 11)
            pdf.set_text_color(0, 0, 0)
            label = "YOU:"
        else:
            pdf.set_font("Arial", "B", 11)
            pdf.set_text_color(100, 100, 100) # Gray for Chatbot
            label = "CHATBOT:"

        pdf.multi_cell(0, 8, txt=label)
        pdf.set_font("Arial", "", 11)
        pdf.set_text_color(0, 0, 0)
        # multi_cell handles line wrapping automatically
        pdf.multi_cell(0, 8, txt=content.replace("**You:** ", "").replace("**Chatbot:** ", ""))
        pdf.ln(5)
        pdf.line(10, pdf.get_y(), 200, pdf.get_y())
        pdf.ln(5)

    filename = f"deepsite_report_{datetime.now().strftime('%H%M%S')}.pdf"
    pdf.output(filename)
    return filename



# --- 4. Helper Function: Clear History ---
def clear_chat():
    # Returns the initial welcome message to reset the Chatbot component
    return [{"role": "assistant", "content": "**🤖 Chatbot:** 👋 Welcome! History has been cleared. How can I help you now? 🚀"}]

# --- 5. Helper Function: Chat Wrapper with Explicit Labels ---
def chat_ui_wrapper(message, history):
    # Retrieve answer from your RAG logic (Assumes rag_system is initialized in previous cells)
    try:
        res = rag_system.ask(message)
        answer = res['answer']
        routing = res['routing']
        pages = ', '.join(map(str, set(res['pages'])))
    except Exception as e:
        answer = f"Error: Ensure Section 4-6 were run correctly. ({str(e)})"
        routing = "Error"
        pages = "N/A"

    # Format the Source Attribution "Trust Signal"
    source_html = f"<div class='source-box'>🧐 <b>Trust Signal:</b> Verified from {routing} (Pages: {pages})</div>"

    # 1. Append User Input with "You" label
    history.append({"role": "user", "content": f"**👤 You:** {message}"})

    # 2. Append AI Response with "Chatbot" label
    bot_response = f"**🤖 Chatbot:** {answer}\n\n{source_html}"
    history.append({"role": "assistant", "content": bot_response})

    return "", history

# --- 6. Custom CSS for "Deepsite" Monochrome Aesthetic ---
custom_css = """
.welcome-text { text-align: center; margin-bottom: 25px; }
.welcome-text h1 { font-family: 'Courier New', monospace; font-weight: bold; margin-bottom: 0px; }
.welcome-text h3 { color: #4b5563; margin-top: 5px; margin-bottom: 2px; }
.welcome-text p { color: #6b7280; font-size: 0.95em; margin-top: 5px; }
.chatbot-container { border: 2px solid #000 !important; border-radius: 12px !important; background-color: #ffffff !important; }
.source-box { background-color: #f9f9f9; border-left: 5px solid #000; padding: 12px; margin-top: 15px; font-size: 0.85em; }
.status-window { font-family: 'Courier New', monospace; font-size: 0.9em; background-color: #f3f4f6 !important; border: 1px solid #000 !important; }
"""

# --- 7. Building the UI Layout ---
with gr.Blocks(theme=gr.themes.Monochrome(), css=custom_css) as demo:

    # Header Section
    with gr.Column(elem_classes="welcome-text"):
        gr.Markdown("# AI-Powered Document Intelligence Chatbot")
        gr.Markdown("### Providing assistance with document search. ✨")
        gr.Markdown("Upload document and enter search request in chatbot")

    with gr.Tabs():
        # TAB 1: Chat interface
        with gr.Tab("💬 Chat Interface"):
            chatbot = gr.Chatbot(
                value=[{"role": "assistant", "content": "**Chatbot:** 👋 Welcome! Upload files in the next tab to begin. 🚀"}],
                height=500,
                type="messages",
                elem_classes="chatbot-container",
                allow_tags=True
            )
            with gr.Row():
                msg_input = gr.Textbox(placeholder="Ask a question about your docs...", scale=7, container=False)
                send_btn = gr.Button("Send 🚀", variant="primary", scale=1)

            with gr.Row():
                clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary")
                export_btn = gr.Button("📄 Export PDF Report")
                download_link = gr.File(label="📥 Click to Download PDF", scale=1, interactive=False)

        # TAB 2: Upload and Guidance
        with gr.Tab("📂 Upload & Guidance"):
            gr.Markdown("""
            ### 🛠️ Pro-Tips for Best Results
            For best results, always upload clean, high-quality documents so the system can process them accurately.
            Use document type filters to narrow your search and avoid irrelevant results.
            If your query is broad, try breaking it into smaller, more specific questions — this often leads to more accurate answers.
            **And remember, the magic happens when your data is well-prepared and your questions are clear, because even the smartest AI needs good input to give great output.**
            """)

            file_in = gr.File(label="Upload your document 📂", file_types=[".pdf"])
            ingest_btn = gr.Button("Initialize AI Index ✨", variant="primary")

            log_out = gr.Textbox(
                label="System Status & Metadata",
                lines=6,
                elem_classes="status-window",
                placeholder="Technical details will appear here after upload..."
            )

    # ---8. Event Wiring ---

    # Send button & Enter key functionality
    send_btn.click(chat_ui_wrapper, [msg_input, chatbot], [msg_input, chatbot])
    msg_input.submit(chat_ui_wrapper, [msg_input, chatbot], [msg_input, chatbot])

    # Clear Wiring
    clear_btn.click(clear_chat, outputs=chatbot)

    # PDF Generation Wiring: Button triggers the function, output goes to the File component
    export_btn.click(
        fn=export_chat_pdf,
        inputs=chatbot,
        outputs=download_link
        )

    # Ingest button functionality with Metadata first
    # Sequence: 1. Show processing status -> 2. Process -> 3. Show success with stats
    ingest_btn.click(
        fn=get_file_metadata,
        inputs=file_in,
        outputs=log_out
    ).then(
        fn=lambda f: rag_system.ingest(f.name),
        inputs=file_in,
        outputs=None
    ).then(
        fn=get_final_status,
        inputs=file_in,
        outputs=log_out
    )

# Launch the Deepsite App
demo.launch(debug=True, share=True)




